<a href="https://colab.research.google.com/github/ehdob-cid20/data-science-clothing-reviews/blob/main/clothing_reviews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1><strong>Análise de sentimentos em avaliações de e-commerce de roupas</strong></h1>


**Objetivo**:

Classificar as avaliações de e-commerce de roupas em positiva, negativa ou neutra.

*   Dataset: https://www.kaggle.com/nicapotato/womens-ecommerce-clothing-reviews

***Orientador:*** Thiago de Melo

***Orientando:*** Edson Brilhante

# Importando as bibliotecas

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Importando o *dataset*

In [22]:
dataset = pd.read_csv('/content/drive/MyDrive/clothing-reviews/Womens Clothing E-Commerce Reviews.csv')

# Visualização do *dataset*

In [23]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23486 entries, 0 to 23485
Data columns (total 11 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   Unnamed: 0               23486 non-null  int64 
 1   Clothing ID              23486 non-null  int64 
 2   Age                      23486 non-null  int64 
 3   Title                    19676 non-null  object
 4   Review Text              22641 non-null  object
 5   Rating                   23486 non-null  int64 
 6   Recommended IND          23486 non-null  int64 
 7   Positive Feedback Count  23486 non-null  int64 
 8   Division Name            23472 non-null  object
 9   Department Name          23472 non-null  object
 10  Class Name               23472 non-null  object
dtypes: int64(6), object(5)
memory usage: 2.0+ MB


# Modelagem

## Seleciona as colunas '*Review Text*' e '*Rating*'

In [24]:
dataset = dataset[['Review Text', 'Rating']]
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23486 entries, 0 to 23485
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Review Text  22641 non-null  object
 1   Rating       23486 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 367.1+ KB


## Encoding

Seleciona o atributo *'Rating'* e o converte da seguinte forma:



*   1 e 2: negativa
*   3: neutra
*   4 e 5: positiva

In [26]:
dataset['Rating'] = dataset['Rating'].apply(lambda x: 'positiva' if x >= 4 else ('neutra' if x == 3 else 'negativa'))
dataset['Rating'] = pd.Categorical(dataset['Rating'])

In [28]:
dataset

,Review Text,Rating
0,Absolutely wonderful - silky and sexy and comf...,positiva
1,Love this dress! it's sooo pretty. i happene...,positiva
2,I had such high hopes for this dress and reall...,neutra
3,"I love, love, love this jumpsuit. it's fun, fl...",positiva
4,This shirt is very flattering to all due to th...,positiva
...,...,...
23481,I was very happy to snag this dress at such a ...,positiva
23482,"It reminds me of maternity clothes. soft, stre...",neutra
23483,"This fit well, but the top was very see throug...",neutra
23484,I bought this dress for a wedding i have this ...,neutra
